In [3]:
!pip install langchain langchain-google-genai python-dotenv langchain-groq

### Load API Key

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

### Use Models

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=gemini_api_key,
    max_retries=1,
)

/home/sudarshan/Desktop/Workshop/Code/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
response = model.invoke("Hello, What is use of GPU in Deep learing, explain in 3 sentences")

In [7]:
print(response.content)

GPUs are highly efficient at parallel processing, meaning they can perform thousands of computations simultaneously. This capability is essential for deep learning because training neural networks involves an immense number of repetitive matrix multiplications and additions across vast datasets. Consequently, GPUs drastically accelerate model training, enabling researchers and practitioners to build and experiment with much larger, more complex models in a fraction of the time.


In [8]:
response.usage_metadata

{'input_tokens': 18,
 'output_tokens': 246,
 'total_tokens': 264,
 'input_token_details': {'cache_read': 0},
 'output_token_details': {'reasoning': 174}}

### Context aware generation

In [9]:
system_prompt = """
You are a helpful and polite support assistant for **Lumbini ICT Campus**.

Your role is to assist students by answering questions **only related to Lumbini ICT Campus**.
You may respond to simple greetings.

If a user asks any question that is **not related to the campus**, clearly reply:
"I don't know."

If a user asks for information about the campus that is **not included below**, politely ask them to contact the college directly at:
+977 9812345678

---

### Campus Information

**Location**
- Kaligandaki Chowk, Gaindakot-5, Nawalpur

**Available Courses**
- BSc. CSIT
- BCA
- BIM
- BHM

**Course Details**
- **BSc. CSIT**: 4 years, only students with a science background in +2
- **BCA**: 4 years, students from any background
- **BIM**: 3 years, only students with a management background
- **BHM**: 4 years, students from any background

**Campus Advantages**
- Regular extra classes
- Workshops
- Sports events
- Hackathons
"""

In [10]:
from langchain.messages import HumanMessage, SystemMessage


query = input("Ask anything about campus")

response = model.invoke(
    [
        SystemMessage(content=system_prompt),
        HumanMessage(query)
    ]
)

In [11]:
response.content

'The BCA course at Lumbini ICT Campus is a 4-year program and is open to students from any academic background.'

### Tool calling

In [12]:
from langchain.tools import tool
import json

@tool
def search_database():
    """
    Search the products database to see records of `products`
    """
    records = [
        {
            "id": 1,
            "name": "Phone Case",
            "price": 800,
            "available": True
        },
        {
            "id": 2,
            "name": "Charger",
            "price": 3200,
            "available": True
        },
        {
            "id": 3,
            "name": "Power bank",
            "price": 6000,
            "available": False,
            "next_available_date": "2026-01-01"
        }
    ]
    return json.dumps(records)

In [13]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    tools=[search_database]
)

In [14]:
from langchain.messages import HumanMessage


messages = [HumanMessage("hi")]

response = agent.invoke({"messages": messages})

In [15]:
response["messages"][-1]

AIMessage(content='Hello! I can help you search the product database. What are you looking for?', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b4a92-108d-7fb1-b036-a5e8ab727fe0-0', usage_metadata={'input_tokens': 34, 'output_tokens': 17, 'total_tokens': 51, 'input_token_details': {'cache_read': 0}})

In [16]:
response = agent.invoke({"messages": HumanMessage("What is price of charger")})

In [17]:
response

{'messages': [HumanMessage(content='What is price of charger', additional_kwargs={}, response_metadata={}, id='2c859f06-fe59-47c5-a24a-8a38d6351ef5'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_database', 'arguments': '{}'}, '__gemini_function_call_thought_signatures__': {'f48cabfe-8b2c-4c98-a67d-faf06bdeb576': 'CrISAXLI2nyQbFuZoVnWfZkD/yNjEGalBUy27LkQYrdevtG3GXTFMh+akKvRP9m1k6rEQaULfWPSSHE7+Y6q9kT3XCpc9uLjP4ZMCBdHaoR8RohDidDkYVF1WANEqunceqcYLTz37oTWaltNqHIVQ2dT5TLSKVVdcjm1PpByoO/ZYaoG1KnTYAvNQ7SlI13KSM+XRCluQ1xFXfidOQx++KVukECLJ4Yl5tb7dV2c2H1UfrUUeNI4BSc3V2joPGYl8dW5U8NPSi+ZxjOwfXOuSe7QuzPKyWKz+viA7QxmN5YgvnAF6jo4gV7bg+VfudqXcFCs0S63OXLRjlx5MIXW6sfWT5R5CDSwgbN6PonHIC+N2qJXS7ZkzlWoD7tZIcKHDulYnTOTFuAw9wmAiiMT9xvFBhgWkiVezQUW7DAgt1gEhk1hXaSW5psa18MZ7kB7JJIARwt/fGni/0V7ax7Ug9Ow9AC5m4Khu9y7WYm2EXeoreLqf4lKBC3N3HuiBtFysvKGfcQ/iBXDmQnYGqvOjIeVUrs4sK6swF7PyaqGAHthJOovRWWy76gAdttu6eJtUKP7p7HIEBS+TRC5lmTr+LJ4qZr70Ck/1kQ17qRHPcnCKy3wbFvChkHVQxwXOh3jBzAPgLF7Fgav

In [18]:
response["messages"][-1].content

'The price of the Charger is 3200.'

In [19]:
agent.invoke({"messages": "Who is current PM of USA"})

{'messages': [HumanMessage(content='Who is current PM of USA', additional_kwargs={}, response_metadata={}, id='66e9bc4c-3d41-496e-9d29-a17b103325ef'),
  AIMessage(content=[{'type': 'text', 'text': 'I cannot answer this question. There is no "PM" (Prime Minister) in the USA; the leader of the USA is the President. My knowledge base does not have the current head of state for any country.', 'extras': {'signature': 'Cs4FAXLI2nwQfMeOF4u7cGvzdACXi/siNTE1+nEAMQSOwAEkBKHtXm9hNVmpNZPVTMz99b56ys0fPHjb4AJZl0USxHeQyXPMZ8wvfV178tKDwBdoIATuyByzokyrtiPUB5OIpJRFlkV/hdvUrYaEVAoiemyP/ED+sIxvcx/F4E2N+RD88Qk8OtCz/5ANnEn7lRVmQ3YA5TM0oNeJPWXwlvm25slxJKTuMZZm7fgAr2uHvlJEAguoR1wS9rQyvyqziV8tQ/yHxP92+7pUeqMD2Rho5JwYFqEMc1eZPP5+dBdYvxws1VQ92Byk+KfuVQtv1r+teeHkTm+rwjlpSpQnyC7d9P6oeEH6c+NvZchK/kXJuJ+eDe6EzoQaH7mhmRSAlrdYL5buIEr+xH9uwVI2fJraAG9MbNpvLXVln1IeBmTPN/Xd0Xeszgqcm4TfTJ403UuhfoA2WXgcCUqQD2MQIqxOU38KyaYlt+PyUlSVA9hzR5C1y5xwT2AGzhqw0Vatdjrv0BtL8fLyF52RLmYf00ftYk8+GstFv4rPPKD+cch31DkXtcpUNLTabpoLBF+oyakaaVj